1. setup

Importera in bibliotek
Sätt k-värde 
Sät top_n-värde

In [191]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors



Nedan lägger vi till csv-filerna med Användares betyg och spelen 
....

In [154]:
df_ratings = pd.read_csv("dataset/user_ratings.csv")
df_games = pd.read_csv("dataset/games.csv")
df_ratings.info()
#df_games.info()
df = pd.merge(df_ratings, df_games[['BGGId', 'Name']], on='BGGId', how='left')
df.info()   


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18942215 entries, 0 to 18942214
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 433.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18942215 entries, 0 to 18942214
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
 3   Name      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 722.6+ MB


In [155]:
df_ratings

,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP
...,...,...,...
18942210,165521,3.0,rseater
18942211,165521,3.0,Bluefox86
18942212,165521,3.0,serginator
18942213,193488,1.0,CaptainCattan


In [156]:
df.describe()

,BGGId,Rating
count,1.894222e+07,1.894222e+07
mean,1.099533e+05,7.130957e+00
std,9.280217e+04,1.543023e+00
min,1.000000e+00,1.000000e-04
25%,1.611000e+04,6.000000e+00
50%,1.075290e+05,7.000000e+00
75%,1.813040e+05,8.000000e+00
max,3.491610e+05,1.000000e+01


2 Preprocessing

Handle missing valuee

In [157]:
any_null_values = df.isnull().any()
any_null_values

null_counts = df.isnull().sum()
print(null_counts)

BGGId        0
Rating       0
Username    63
Name         0
dtype: int64


In [158]:
df = df.dropna()

In [159]:
df.isnull().any()

BGGId       False
Rating      False
Username    False
Name        False
dtype: bool

In [160]:
df

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942210,165521,3.0,rseater,Rory's Story Cubes: Medic
18942211,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942212,165521,3.0,serginator,Rory's Story Cubes: Medic
18942213,193488,1.0,CaptainCattan,Airlines


In [161]:
duplicates = df.duplicated().value_counts()
duplicates

False    18942152
dtype: int64

In [162]:
duplicates.describe()

count           1.0
mean     18942152.0
std             NaN
min      18942152.0
25%      18942152.0
50%      18942152.0
75%      18942152.0
max      18942152.0
dtype: float64

Remove duplicate ratings for the same game by the same user (due to different version of the game etc)

In [163]:
# Calculate the mean rating for each group of duplicate rows
mean_ratings = df.groupby(['Username', 'BGGId'])['Rating'].mean()

In [164]:
mean_ratings

Username        BGGId 
 Fu_Koios       112092    9.0
                223033    9.0
 beastvol       13        8.0
                118       7.0
                278       7.0
                         ... 
Æleksandr Þræð  2399      7.0
                2932      6.0
                5451      8.0
                5554      7.0
                9962      8.0
Name: Rating, Length: 18909465, dtype: float64

In [165]:
# Filter the DataFrame for the specific user and item
specific_user_item_ratings = df[(df['Username'] == 'hariseldoncwb') & (df['BGGId'] == 135382)]

# Calculate the mean rating for the specific user and item
mean_rating_specific_user_item = specific_user_item_ratings['Rating'].mean()
mean_rating_specific_user_item

5.5

In [166]:
# Merge the mean ratings back to the original DataFrame based on 'Username' and 'Name'
df_removed_duplicates = df.merge(mean_ratings, on=['Username', 'BGGId'], suffixes=('', '_mean'))

In [167]:
df_removed_duplicates

,BGGId,Rating,Username,Name,Rating_mean
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,8.0
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition,8.0
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition,8.0
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition,8.0
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition,8.0
...,...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic,3.0
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic,3.0
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic,3.0
18942150,193488,1.0,CaptainCattan,Airlines,1.0


In [168]:
# Filter the DataFrame to keep only the original rows (non-duplicates) and the calculated mean ratings
df_unique = df_removed_duplicates[~df_removed_duplicates.duplicated(subset=['Username', 'BGGId'], keep=False)]

In [169]:
df_unique

,BGGId,Rating,Username,Name,Rating_mean
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,8.0
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition,8.0
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition,8.0
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition,8.0
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition,8.0
...,...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic,3.0
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic,3.0
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic,3.0
18942150,193488,1.0,CaptainCattan,Airlines,1.0


In [170]:
# Drop the original 'Rating' column and rename the mean rating column
df_dropped = df_unique.drop(columns=['Rating']).rename(columns={'Rating_mean': 'Rating'})

In [171]:
# Reindex the columns to move 'Rating' to the second position
column_order = ['BGGId', 'Rating', 'Username', 'Name']
df_dropped = df_dropped.reindex(columns=column_order)

In [172]:
df_dropped

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic
18942150,193488,1.0,CaptainCattan,Airlines


In [173]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df_dropped.groupby('BGGId').size()
grouped_counts

BGGId
1          5295
2           560
3         15077
4           342
5         18152
          ...  
347146       42
347521       71
348955       67
349131       66
349161        7
Length: 21925, dtype: int64

Filter dataset

In [174]:
game_ratings = 50

rating_counter = df_dropped['BGGId'].value_counts()

filtered_out = rating_counter[rating_counter < game_ratings].index

games_mask = df_dropped['BGGId'].isin(filtered_out)
#games_mask = df['BGGId'].isin(rating_counter[rating_counter < game_ratings].index)
#games_mask
df_dropped.drop(index=df_dropped[games_mask].index, inplace=True)

df_dropped

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic
18942150,193488,1.0,CaptainCattan,Airlines


In [175]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df_dropped.groupby('BGGId').size()
grouped_counts

BGGId
1          5295
2           560
3         15077
4           342
5         18152
          ...  
346703     1000
346965      253
347521       71
348955       67
349131       66
Length: 17543, dtype: int64

In [176]:
# Group by 'Username' and count the number of rows for each group
grouped_user_counts = df_dropped.groupby('Username').size()
grouped_user_counts

Username
 Fu_Koios              2
 beastvol              9
 mycroft              14
 woh                   5
(mostly) harmless      1
                    ... 
zzzuzu                39
zzzvone               21
zzzxxxyyy             32
zzzzzane             154
Æleksandr Þræð        12
Length: 408125, dtype: int64

In [177]:
user_ratings = 20

rating_counter = df_dropped['Username'].value_counts()

filtered_out = rating_counter[rating_counter < user_ratings].index

user_mask = df_dropped['Username'].isin(filtered_out)

df_dropped.drop(index=df_dropped[user_mask].index, inplace=True)

df_dropped

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942146,165521,4.0,crito13,Rory's Story Cubes: Medic
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic


In [178]:
# Group by 'Username' and count the number of rows for each group
grouped_user_counts = df_dropped.groupby('Username').size()
grouped_user_counts

Username
-=Yod@=-       304
-Johnny-       474
-Loren-         79
-LucaS-         23
-Morphling-     51
              ... 
zzzoren        182
zzzuzu          39
zzzvone         21
zzzxxxyyy       32
zzzzzane       154
Length: 168314, dtype: int64

Train test split

In [179]:
# Define a function to split the data for each user
def user_split(data, test_size=0.2):    
    train_to_concat = []
    test_to_concat = []
    for user, group in data.groupby('Username'):
        train_user, test_user = train_test_split(group, test_size=test_size, train_size=1-test_size)
        train_to_concat.append(train_user)
        test_to_concat.append(test_user)
    
    train = pd.concat(train_to_concat, ignore_index=True)
    test = pd.concat(test_to_concat, ignore_index=True)

    return train, test

# Split the data
train_set, test_set = user_split(df_dropped, test_size=0.2)


In [180]:
train_set

,BGGId,Rating,Username,Name
0,250442,7.0,-=Yod@=-,Crypt
1,186659,7.5,-=Yod@=-,Tiny Epic Galaxies: Deluxe Edition
2,50,8.5,-=Yod@=-,Lost Cities
3,188866,6.5,-=Yod@=-,Awkward Guests
4,85250,7.0,-=Yod@=-,The Dwarf King
...,...,...,...,...
13823690,111124,7.0,zzzzzane,Dark Moon
13823691,169786,7.5,zzzzzane,Scythe
13823692,164265,7.0,zzzzzane,Witness
13823693,134352,7.0,zzzzzane,Two Rooms and a Boom


In [181]:
test_set

,BGGId,Rating,Username,Name
0,117854,6.5,-=Yod@=-,Kosmonauts
1,34707,7.0,-=Yod@=-,The Hanging Gardens
2,226518,7.0,-=Yod@=-,Exit: The Game – The Sunken Treasure
3,10630,9.0,-=Yod@=-,Memoir '44
4,150996,6.5,-=Yod@=-,Lords of War: Templars versus Undead
...,...,...,...,...
3539950,323613,7.5,zzzzzane,LUNA Capital
3539951,144344,8.0,zzzzzane,Rococo
3539952,73439,8.0,zzzzzane,Troyes
3539953,284083,10.0,zzzzzane,The Crew: The Quest for Planet Nine


filtrera dataframe 

3 Similarity Calculation

 Skapa en matris av item-för-item-similarity baserat på cosine similarity

In [182]:
# Assuming df is your pandas DataFrame with columns BGGId, Rating, Username, Name
# Pivot the DataFrame so that rows represent users and columns represent items
pivot_df = train_set.pivot(index='Username', columns='BGGId', values='Rating').fillna(0)



/var/folders/f3/n44hzk017jg3wr6ylhv9l8lr0000gn/T/ipykernel_91836/3450243160.py:3: PerformanceWarning: The following operation may generate 2952732502 cells in the resulting pandas object.
  pivot_df = train_set.pivot(index='Username', columns='BGGId', values='Rating').fillna(0)


In [183]:
pivot_df

BGGId,1,2,3,4,5,6,7,8,9,10,...,344768,345036,345584,346482,346501,346703,346965,347521,348955,349131
Username,,,,,,,,,,,,,,,,,,,,,
-=Yod@=-,0.0,0.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Johnny-,5.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Loren-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-LucaS-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Morphling-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzzoren,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzzuzu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzzvone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [185]:
# Transpose the user-item matrix to obtain an item-user matrix
item_user_matrix = pivot_df.T

In [186]:
item_user_matrix

Username,-=Yod@=-,-Johnny-,-Loren-,-LucaS-,-Morphling-,-mIDE-,-mik-,-pj-,-toni-,-xXx-,...,zzyzxuk,zzzabiss,zzzeagle,zzzhsm,zzzkardel,zzzoren,zzzuzu,zzzvone,zzzxxxyyy,zzzzzane
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,4.0,0.0,0.0,0.0,8.0,0.0,0.0,9.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
346965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
347521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [187]:
# Compute cosine similarity between items
item_similarity = cosine_similarity(item_user_matrix, dense_output=False)



In [188]:
item_similarity

array([[1.        , 0.04224476, 0.23615647, ..., 0.00357582, 0.        ,
        0.02111222],
       [0.04224476, 1.        , 0.05627787, ..., 0.00600601, 0.        ,
        0.        ],
       [0.23615647, 0.05627787, 1.        , ..., 0.01393259, 0.00442959,
        0.01838933],
       ...,
       [0.00357582, 0.00600601, 0.01393259, ..., 1.        , 0.00244807,
        0.        ],
       [0.        , 0.        , 0.00442959, ..., 0.00244807, 1.        ,
        0.        ],
       [0.02111222, 0.        , 0.01838933, ..., 0.        , 0.        ,
        1.        ]])

In [189]:
# Create a DataFrame with item names as both rows and columns
item_item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix.index, columns=item_user_matrix.index)

# Now item_item_similarity_df is your item-item similarity matrix with item names as both row and column labels

In [206]:
item_item_similarity_df

BGGId,1,2,3,4,5,6,7,8,9,10,...,344768,345036,345584,346482,346501,346703,346965,347521,348955,349131
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.042245,0.236156,0.110157,0.245028,0.051265,0.095172,0.075352,0.166468,0.186407,...,0.014651,0.012133,0.008354,0.017105,0.016574,0.020744,0.013029,0.003576,0.000000,0.021112
2,0.042245,0.000000,0.056278,0.052978,0.073469,0.033542,0.064748,0.034108,0.051368,0.065613,...,0.000000,0.000000,0.000000,0.000000,0.006394,0.004259,0.000000,0.006006,0.000000,0.000000
3,0.236156,0.056278,0.000000,0.083479,0.279624,0.035158,0.124568,0.043962,0.157062,0.239486,...,0.021121,0.014278,0.010166,0.012980,0.015752,0.031575,0.028952,0.013933,0.004430,0.018389
4,0.110157,0.052978,0.083479,0.000000,0.083748,0.143612,0.089438,0.129525,0.146013,0.114824,...,0.003602,0.011108,0.000000,0.020700,0.030562,0.014934,0.014835,0.000000,0.000000,0.030070
5,0.245028,0.073469,0.279624,0.083748,0.000000,0.037900,0.139019,0.057608,0.135014,0.221985,...,0.014726,0.014614,0.009744,0.014583,0.008907,0.025560,0.022768,0.013641,0.002519,0.010941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346703,0.020744,0.004259,0.031575,0.014934,0.025560,0.000000,0.017211,0.005496,0.018335,0.034212,...,0.009214,0.026771,0.018521,0.044227,0.035461,0.000000,0.044462,0.000000,0.000000,0.047510
346965,0.013029,0.000000,0.028952,0.014835,0.022768,0.000000,0.011379,0.009311,0.023415,0.033920,...,0.079830,0.037451,0.034106,0.063336,0.081484,0.044462,0.000000,0.000000,0.000000,0.146388
347521,0.003576,0.006006,0.013933,0.000000,0.013641,0.000000,0.005891,0.000000,0.006905,0.006911,...,0.019054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002448,0.000000


In [ ]:
# Set similarity to 0 for each item to itself
np.fill_diagonal(item_item_similarity_df.values, 0)

In [215]:
# Convert similarity to distances (inversely proportional to similarities)
distance_matrix = 1 - item_item_similarity_df

In [216]:
distance_matrix

BGGId,1,2,3,4,5,6,7,8,9,10,...,344768,345036,345584,346482,346501,346703,346965,347521,348955,349131
BGGId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.957755,0.763844,0.889843,0.754972,0.948735,0.904828,0.924648,0.833532,0.813593,...,0.985349,0.987867,0.991646,0.982895,0.983426,0.979256,0.986971,0.996424,1.000000,0.978888
2,0.957755,1.000000,0.943722,0.947022,0.926531,0.966458,0.935252,0.965892,0.948632,0.934387,...,1.000000,1.000000,1.000000,1.000000,0.993606,0.995741,1.000000,0.993994,1.000000,1.000000
3,0.763844,0.943722,1.000000,0.916521,0.720376,0.964842,0.875432,0.956038,0.842938,0.760514,...,0.978879,0.985722,0.989834,0.987020,0.984248,0.968425,0.971048,0.986067,0.995570,0.981611
4,0.889843,0.947022,0.916521,1.000000,0.916252,0.856388,0.910562,0.870475,0.853987,0.885176,...,0.996398,0.988892,1.000000,0.979300,0.969438,0.985066,0.985165,1.000000,1.000000,0.969930
5,0.754972,0.926531,0.720376,0.916252,1.000000,0.962100,0.860981,0.942392,0.864986,0.778015,...,0.985274,0.985386,0.990256,0.985417,0.991093,0.974440,0.977232,0.986359,0.997481,0.989059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346703,0.979256,0.995741,0.968425,0.985066,0.974440,1.000000,0.982789,0.994504,0.981665,0.965788,...,0.990786,0.973229,0.981479,0.955773,0.964539,1.000000,0.955538,1.000000,1.000000,0.952490
346965,0.986971,1.000000,0.971048,0.985165,0.977232,1.000000,0.988621,0.990689,0.976585,0.966080,...,0.920170,0.962549,0.965894,0.936664,0.918516,0.955538,1.000000,1.000000,1.000000,0.853612
347521,0.996424,0.993994,0.986067,1.000000,0.986359,1.000000,0.994109,1.000000,0.993095,0.993089,...,0.980946,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.997552,1.000000


Calculate nearest neighbor

In [218]:
k = 20

# Create a NearestNeighbors object
nearest_neighbors_model = NearestNeighbors(n_neighbors=k, metric='precomputed')

In [219]:
# Fit the model using the item-item similarity matrix
nearest_neighbors_model.fit(distance_matrix.values)

NearestNeighbors(metric='precomputed', n_neighbors=20)

In [245]:
BGGId_80006 = 80006
BGGId_2 = 2

# Find the index of the item
item_index = distance_matrix.index.get_loc(BGGId_2)

In [246]:
item_index

1

In [247]:
# Find the indices of the nearest neighbors
distances, neighbor_indices = nearest_neighbors_model.kneighbors([distance_matrix.iloc[item_index]], return_distance=True)

In [248]:
distances

array([[0.87657606, 0.90033503, 0.90437225, 0.90704539, 0.91288778,
        0.92010317, 0.92037898, 0.92104781, 0.92106783, 0.92178282,
        0.92219941, 0.92239426, 0.92273396, 0.92311167, 0.92382116,
        0.92387254, 0.9241849 , 0.92426584, 0.92506973, 0.92522267]])

In [249]:
neighbor_indices

array([[  28,  652,  296,  802, 1332,  627,   84,   97,  170, 6697,  902,
         263, 2613,  717, 3283,  411,   90,   92,  525, 1803]])

In [250]:
#BGGId_item_1 = 2
#BGGId_item_2 = 9830

# Retrieve the indices of the items in the DataFrame
#item1_index = item_item_similarity_df.index.get_loc(BGGId_item_1)  # Replace 'item2' with the name of the first item
#item2_index = item_item_similarity_df.index.get_loc(BGGId_item_2)   # Replace '3981' with the name of the second item

# Retrieve the distance between the two items from the distance matrix
distance_between_items = distance_matrix.iloc[1, 28]
distance_between_items

0.8765760649125957

In [251]:
item_item_similarity_between_items = item_item_similarity_df.iloc[1, 28]
item_item_similarity_between_items

0.1234239350874043

Här nedan skriver vi funktionen som tar input data, user, k, top_n

Nedan skriver vi funktion som visar resultat av rekomendation i form av en sorts lista baserat på recommender funktionen

Nedan här skriver vi evalueringsfunktioner av modellen och utskrift 